In [1]:
from dotenv import find_dotenv, load_dotenv
from langfuse import get_client

_ = load_dotenv(find_dotenv())
langfuse = get_client()

In [2]:
# get TraceWithDetails
traces = langfuse.api.trace.list(limit=1)
trace_data = traces.data[0]
trace_id = trace_data.id
print(f"trace_id: {trace_data.id}")
print(f"trace input: {trace_data.input}")
print(f"trace data type: {type(trace_data)}")

trace_id: fb8f8238ea13c59aab69612f3a4cafd1
trace input: None
trace data type: <class 'langfuse.api.resources.commons.types.trace_with_details.TraceWithDetails'>


In [3]:
trace_data

TraceWithDetails(id='fb8f8238ea13c59aab69612f3a4cafd1', timestamp=datetime.datetime(2025, 9, 19, 16, 40, 50, 697000, tzinfo=datetime.timezone.utc), name='day4_test', input=None, output=None, session_id=None, release=None, version=None, user_id=None, metadata={'resourceAttributes': {'telemetry.sdk.language': 'python', 'telemetry.sdk.name': 'opentelemetry', 'telemetry.sdk.version': '1.37.0', 'service.name': 'unknown_service'}, 'scope': {'name': 'langfuse-sdk', 'version': '3.4.0', 'attributes': {'public_key': 'pk-lf-44100c08-817a-4a6a-85d7-6f36553c3d8c'}}}, tags=[], public=False, environment='default', html_path='/project/cmfp3pmsa002sad08w7g4l78h/traces/fb8f8238ea13c59aab69612f3a4cafd1', latency=9.387, total_cost=0.001365999999, observations=['0febcc3ea921afdf', '2bfb479428418b3f', 'b02175264340c44d', '0dcfd802fab66951', 'e5c7d14c1df50d76', 'ff89ae8e570e0949', '94a708135d59ae4b', '9d14eae1672a5566', 'f802ee04b7f24f80', '60511bda70778bd7', '6f585f07613642b2', '4c58117a024b1eea', 'f0031e28

In [4]:
TraceWithFullDetails = langfuse.api.trace.get(trace_id)
TraceWithFullDetails

TraceWithFullDetails(id='fb8f8238ea13c59aab69612f3a4cafd1', timestamp=datetime.datetime(2025, 9, 19, 16, 40, 50, 697000, tzinfo=datetime.timezone.utc), name='day4_test', input=None, output=None, session_id=None, release=None, version=None, user_id=None, metadata={'resourceAttributes': {'telemetry.sdk.language': 'python', 'telemetry.sdk.name': 'opentelemetry', 'telemetry.sdk.version': '1.37.0', 'service.name': 'unknown_service'}, 'scope': {'name': 'langfuse-sdk', 'version': '3.4.0', 'attributes': {'public_key': 'pk-lf-44100c08-817a-4a6a-85d7-6f36553c3d8c'}}}, tags=[], public=False, environment='default', html_path='/project/cmfp3pmsa002sad08w7g4l78h/traces/fb8f8238ea13c59aab69612f3a4cafd1', latency=9.387, total_cost=0.001365999999, observations=[ObservationsView(id='0dcfd802fab66951', trace_id='fb8f8238ea13c59aab69612f3a4cafd1', type='CHAIN', name='RetrieverQueryEngine.query', start_time=datetime.datetime(2025, 9, 19, 16, 40, 50, 697000, tzinfo=datetime.timezone.utc), end_time=datetime.

In [5]:
observations = TraceWithFullDetails.observations
len(observations)

15

In [6]:
observations

[ObservationsView(id='0dcfd802fab66951', trace_id='fb8f8238ea13c59aab69612f3a4cafd1', type='CHAIN', name='RetrieverQueryEngine.query', start_time=datetime.datetime(2025, 9, 19, 16, 40, 50, 697000, tzinfo=datetime.timezone.utc), end_time=datetime.datetime(2025, 9, 19, 16, 41, 0, 84000, tzinfo=datetime.timezone.utc), completion_start_time=None, model=None, model_parameters={}, input='什麼是 LlamaIndex 的instrumentation module?', version=None, metadata={'attributes': {'input.value': '什麼是 LlamaIndex 的instrumentation module?', 'output.value': 'instrumentation module 是 LlamaIndex 用來取代舊有 callbacks 模組的監測/埋點套件（在 v0.10.20 及以後版本提供）。它可對 LlamaIndex 應用進行性能與行為的追蹤與度量，並支援與 Langfuse 的整合，讓團隊能協作式地除錯、分析與迭代應用。整合後會捕捉並呈現詳細的 trace（例如上下文增強與 LLM 查詢流程的細節），可在 Langfuse 的介面中檢視。', 'openinference.span.kind': 'CHAIN'}, 'resourceAttributes': {'telemetry.sdk.language': 'python', 'telemetry.sdk.name': 'opentelemetry', 'telemetry.sdk.version': '1.37.0', 'service.name': 'unknown_service'}, 'scope': {'name': 'openinference.instr

In [7]:
for obv in observations:
    print(obv.name)

RetrieverQueryEngine.query
CompactAndRefine.get_response
RetrieverQueryEngine._query
CompactAndRefine.get_response
CompactAndRefine.synthesize
day4_test
DefaultRefineProgram.__call__
OpenAI.predict
OpenAI.chat
TokenTextSplitter.split_text
TokenTextSplitter.split_text
OpenAIEmbedding.get_query_embedding
OpenAIEmbedding._get_query_embedding
VectorIndexRetriever.retrieve
VectorIndexRetriever._retrieve


In [8]:
def retrieve_parser(obv):
    start_time = obv.start_time
    end_time = obv.end_time
    query_text = obv.input
    output_nodes = obv.output
    num_doc = len(output_nodes)
    
    meta_dict = obv.metadata['attributes']
    docs = []
    for idx in range(num_doc):
        rv = {
            'id': meta_dict[f'retrieval.documents.{idx}.document.id'],
            'content': meta_dict[f'retrieval.documents.{idx}.document.content'],
            'score': meta_dict[f'retrieval.documents.{idx}.document.score'],
        }
        docs.append(rv)
        
    rv = {
        'start_time': start_time,
        'end_time': end_time,
        'query': query_text,
        'output_nodes': output_nodes,
        'num_doc': num_doc,
        'docs': docs
    }
    return rv

def chat_parser(obv):
    start_time = obv.start_time
    end_time = obv.end_time
    model_name = obv.model
    num_message = len(obv.input['messages'])
    attributes = obv.metadata['attributes']
    messages = []
    for idx in range(num_message):
        rv = {
            'role': attributes[f'llm.input_messages.{idx}.message.role'],
            'content': attributes[f'llm.input_messages.{idx}.message.content'],
        }
        messages.append(rv)
    rv = {
        'start_time': start_time,
        'end_time': end_time,
        'model_name': model_name,
        'num_message': num_message,
        'messages': messages,
    }
    return rv

In [9]:
data = {}
for obv in observations:
    name = obv.name
    if name == 'VectorIndexRetriever.retrieve':
        data['retrieve_parsed'] = retrieve_parser(obv)
    elif name == 'OpenAI.chat':
        data['chat_parsed'] = chat_parser(obv)

data

{'chat_parsed': {'start_time': datetime.datetime(2025, 9, 19, 16, 40, 52, 164000, tzinfo=datetime.timezone.utc),
  'end_time': datetime.datetime(2025, 9, 19, 16, 41, 0, 83000, tzinfo=datetime.timezone.utc),
  'model_name': 'gpt-5-mini',
  'num_message': 2,
  'messages': [{'role': 'system',
    'content': "You are an expert Q&A system that is trusted around the world.\nAlways answer the query using the provided context information, and not prior knowledge.\nSome rules to follow:\n1. Never directly reference the given context in your answer.\n2. Avoid statements like 'Based on the context, ...' or 'The context information ...' or anything along those lines."},
   {'role': 'user',
    'content': 'Context information is below.\n---------------------\nThe instrumentation module (available in llama-index v0.10.20 and later) is meant to replace the legacy callbacks module.\n\nLangfuse is an open source LLM engineering platform to help teams collaboratively debug, analyze and iterate on their 